# WFI Reference Pixel Reference File Pipeline Example

This notebook shows how to use the WFI Reference File Pipeline to create reference pixel reference files to submit to CRDS. At this stage, it is in the testing phase, and this notebook is used to assemble the steps of the pipeline to make test data.

Presently, the input files come from TVAC1 .asdf files. These data are located on central storage at:

```
/grp/roman/jsanchez/TVAC1_test_files/
```

# Some imports ... 

In [1]:
from wfi_reference_pipeline.utilities import logging_functions
logging_functions.configure_logging('wfi_refpixel_example')

from wfi_reference_pipeline.referencepixel import referencepixel
from wfi_reference_pipeline.resources.make_dev_meta import MakeDevMeta

import asdf
import glob
import logging
import matplotlib.pyplot as plt
import os
import h5py

# Reference Pixel Files

To make a reference pixel reference file, we need to perform the Improved Roman Reference Correction (Rauscher et al., in prep), We'll use several darks to compute a model of the read noise in the normal pixels that is a linear combination of the reference output, left, and right column pixels (IRRC; Rauscher et al., in prep).  The sums for each exposure are then combined/summed together to create a final model that minimizes the 1/f noise given as
        $Fn = \alpha Fa + \gamma Fl + \zeta Fr$.  
 The coefficients are then saved as a final reference class attribute.  

In [2]:
# Set the directory of the TVAC1 data.
tvac1_dir = '/grp/roman/jsanchez/TVAC1_test_files/NOM_OPS/OTP00639_All_TV1a_R1_MCEB'
logging.info(f'Using directory {tvac1_dir} for TVAC1 data.')

In [3]:
# Grab the files.
dark_files = glob.glob(os.path.join(tvac1_dir, '*WFI01*.asdf'))[0:2]

# Record file names in log.
logging.info('Files used for reference pixel creation:')
for df in dark_files:
    logging.info(f'\t{os.path.basename(df)}')
  

# Meta Data

The RFP utilizes python data classes to construct reference file type dependent meta data for development from the "ref_type" keyword, which is then exported into a dictionary for valid usage with roman data models. 

In [4]:
logging.info("Generating ReferencePixel Metadata")
tmp = MakeDevMeta(ref_type='REFPIX')  # Get the default reference file type specific development meta data.
refpixel_dev_meta = tmp.meta_referencepixel.export_asdf_meta()  # Export the meta data as a dictionary.

In [5]:
print(refpixel_dev_meta)  # Inspect default development meta data.

{'reftype': 'REFPIX', 'pedigree': 'DUMMY', 'description': 'For RFP Development and DMS Build Support.', 'author': 'RFP', 'useafter': <Time object: scale='utc' format='isot' value=2023-01-01T00:00:00.000>, 'telescope': 'ROMAN', 'origin': 'STSCI', 'instrument': {'name': 'WFI', 'detector': 'WFI01'}, 'input_units': Unit("DN"), 'output_units': Unit("DN")}


In [6]:
refpixel_dev_meta.update({'useafter': '2023-01-01T00:00:00.000'})  # Update to the useafter date found on CRDS to replace an existing file.
# Or make a new useafter date.

sca = 'WFI01'  # Make the readnoise reference file for WFI05.
refpixel_dev_meta['instrument'].update({'detector': sca})

In [7]:
# add list of files that will make weights to metadata
refpixel_dev_meta['irrc_weight_filenames']= dark_files


In [8]:
print(refpixel_dev_meta)  # Inspect updated meta data.

{'reftype': 'REFPIX', 'pedigree': 'DUMMY', 'description': 'For RFP Development and DMS Build Support.', 'author': 'RFP', 'useafter': '2023-01-01T00:00:00.000', 'telescope': 'ROMAN', 'origin': 'STSCI', 'instrument': {'name': 'WFI', 'detector': 'WFI01'}, 'input_units': Unit("DN"), 'output_units': Unit("DN"), 'irrc_weight_filenames': ['/grp/roman/jsanchez/TVAC1_test_files/NOM_OPS/OTP00639_All_TV1a_R1_MCEB/TVAC1_NOM_OPS_WFI_TOTAL_NOISE_20231021080847_WFI01_uncal.asdf', '/grp/roman/jsanchez/TVAC1_test_files/NOM_OPS/OTP00639_All_TV1a_R1_MCEB/TVAC1_NOM_OPS_WFI_TOTAL_NOISE_20231021083839_WFI01_uncal.asdf']}


# Make Path for temporary file storage

In [9]:
# make path for temporary sums to save in
tmppath = 'tmp/'
        

We now have the files and folder paths of darks for one detector. We can pass this list of images to the ReferencePixel class to make ASDF reference pixel reference files for Roman.

In [10]:
 
refpix_ref = referencepixel.ReferencePixel(dark_files,  outfile = f'roman_{sca}_refpix.asdf',
                                           meta_data=refpixel_dev_meta)
refpix_ref.make_referencepixel_coeffs(tmppath=None)

# save to .asdf file
refpix_ref.save_referencepixel()


---
---
---
# Convert .hdf5 weights to .asdf

In [11]:
# get list of .hdf5 weight files
weight_files = glob.glob('/grp/roman/GROUND_TESTS/TVAC1/NOM_OPS/OTP00639_All_TV1a_R1_MCEB_IRRCcorr/IRRC_weights/IRRC_weight_hdf5/*.h5')

In [14]:
for weight in weight_files:
    # get name of file and sca
    name = os.path.basename(weight).split('.h5')[0]
    sca = os.path.basename(weight).split('_')[-3]
    print(sca)

    # get dark files that original made the weight file but weren't included in the original meta data...oops 
    dark_sca_files = glob.glob(os.path.join(tvac1_dir, '*' + sca + '*.asdf'))

    logging.info("Generating ReferencePixel Metadata")
    tmp = MakeDevMeta(ref_type='REFPIX')  # Get the default reference file type specific development meta data.
    refpixel_dev_meta = tmp.meta_referencepixel.export_asdf_meta()  # Export the meta data as a dictionary.
    refpixel_dev_meta.update({'useafter': '2023-01-01T00:00:00.000'})  # Update to the useafter date found on CRDS to replace an existing file.
    # Or make a new useafter date.
    det = 1  # Make the readnoise reference file for WFI05.
    refpixel_dev_meta['instrument'].update({'detector': sca})
    # add list of files that will make weights to metadata
    refpixel_dev_meta['irrc_weight_filenames'] = dark_sca_files

    outfile = f'/grp/roman/GROUND_TESTS/TVAC1/NOM_OPS/OTP00639_All_TV1a_R1_MCEB_IRRCcorr/IRRC_weights/IRRC_weights_asdf/{name}.asdf'
    refpix_ref = referencepixel.ReferencePixel(weight, outfile = outfile,
                                           meta_data=refpixel_dev_meta)
    
    refpix_ref.make_hdf5_weights()
    # save to .asdf file
    print('saving as: ', outfile)
    refpix_ref.save_referencepixel()

WFI14
saving as:  /grp/roman/GROUND_TESTS/TVAC1/NOM_OPS/OTP00639_All_TV1a_R1_MCEB_IRRCcorr/IRRC_weights/IRRC_weights_asdf/TVAC1_NOM_OPS_WFI_TOTAL_NOISE_20231021063542_WFI14_irrc_weights.asdf
WFI09
saving as:  /grp/roman/GROUND_TESTS/TVAC1/NOM_OPS/OTP00639_All_TV1a_R1_MCEB_IRRCcorr/IRRC_weights/IRRC_weights_asdf/TVAC1_NOM_OPS_WFI_TOTAL_NOISE_20231021064439_WFI09_irrc_weights.asdf
WFI01
saving as:  /grp/roman/GROUND_TESTS/TVAC1/NOM_OPS/OTP00639_All_TV1a_R1_MCEB_IRRCcorr/IRRC_weights/IRRC_weights_asdf/TVAC1_NOM_OPS_WFI_TOTAL_NOISE_20231021062345_WFI01_irrc_weights.asdf
WFI08
saving as:  /grp/roman/GROUND_TESTS/TVAC1/NOM_OPS/OTP00639_All_TV1a_R1_MCEB_IRRCcorr/IRRC_weights/IRRC_weights_asdf/TVAC1_NOM_OPS_WFI_TOTAL_NOISE_20231021064439_WFI08_irrc_weights.asdf
WFI12
saving as:  /grp/roman/GROUND_TESTS/TVAC1/NOM_OPS/OTP00639_All_TV1a_R1_MCEB_IRRCcorr/IRRC_weights/IRRC_weights_asdf/TVAC1_NOM_OPS_WFI_TOTAL_NOISE_20231021080222_WFI12_irrc_weights.asdf
WFI17
saving as:  /grp/roman/GROUND_TESTS/TVA